In [2]:
import numpy as np



In [3]:
drop_cols = ["sourcePayloadAsBase64", "sourcePayloadAsUTF", 
                 "destinationPayloadAsBase64", "destinationPayloadAsUTF"]
import pandas as pd
filepath='TestbedSunJun13Flows.csv'    
df = pd.read_csv(filepath)
df = df.drop_duplicates()
df = df.drop(columns=drop_cols, errors='ignore')

In [4]:
df

,generated,appName,totalSourceBytes,totalDestinationBytes,totalDestinationPackets,totalSourcePackets,direction,sourceTCPFlagsDescription,destinationTCPFlagsDescription,source,protocolName,sourcePort,destination,destinationPort,startDateTime,stopDateTime,Label
0,3/11/2014 17:35,Unknown_UDP,2633658,0,0,28971,L2R,NaN,NaN,192.168.5.122,udp_ip,5353,224.0.0.251,5353,6/12/2010 23:57,6/13/2010 9:24,Normal
2,3/11/2014 17:35,HTTPWeb,64,128,2,1,L2L,"F,A",R,192.168.2.113,tcp_ip,4191,192.168.5.122,80,6/12/2010 23:57,6/12/2010 23:59,Normal
4,3/11/2014 17:35,HTTPWeb,128,64,1,2,L2R,"F,A","F,A",192.168.2.113,tcp_ip,4192,207.241.148.80,80,6/12/2010 23:57,6/12/2010 23:59,Normal
6,3/11/2014 17:35,HTTPImageTransfer,128,128,2,2,L2R,"F,A","F,A",192.168.2.110,tcp_ip,1864,216.49.88.12,80,6/12/2010 23:57,6/12/2010 23:59,Normal
8,3/11/2014 17:35,SecureWeb,64,0,0,1,L2R,"F,A",NaN,192.168.2.113,tcp_ip,4186,63.245.209.72,443,6/12/2010 23:58,6/12/2010 23:59,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275517,3/11/2014 17:35,HTTPWeb,128,64,1,2,L2R,"F,A","F,A",192.168.2.111,tcp_ip,4430,72.14.204.100,80,6/13/2010 23:58,6/13/2010 23:58,Normal
275520,3/11/2014 17:35,HTTPWeb,128,64,1,2,L2R,"F,A","F,A",192.168.2.111,tcp_ip,4428,125.89.196.18,80,6/13/2010 23:58,6/13/2010 23:58,Normal
275522,3/11/2014 17:35,HTTPWeb,256,0,0,4,L2R,"F,A",NaN,192.168.2.111,tcp_ip,4429,69.163.132.48,80,6/13/2010 23:58,6/13/2010 23:58,Normal
275523,3/11/2014 17:35,HTTPWeb,128,0,0,2,L2R,"F,A",NaN,192.168.2.111,tcp_ip,4431,69.163.132.48,80,6/13/2010 23:58,6/13/2010 23:58,Normal


In [5]:
print(df['Label'].value_counts())

Label
Normal    126430
Attack     10139
Name: count, dtype: int64


In [6]:
import pandas as pd
from sklearn.utils import resample


# Check the unique counts in the label column
print(df['Label'].value_counts())

# Separate classes
class_counts = df['Label'].value_counts()
minority_class = class_counts.idxmin()  # Class with fewer samples
majority_class = class_counts.idxmax()  # Class with more samples


df_minority = df[df['Label'] == minority_class]
df_majority = df[df['Label'] == majority_class]


# Undersample both classes to 70 records each
df_minority_undersampled = resample(df_minority, 
                                    replace=False, 
                                    n_samples=100, 
                                    random_state=42)

df_majority_undersampled = resample(df_majority, 
                                    replace=False, 
                                    n_samples=100, 
                                    random_state=42)

# Combine the undersampled data
df_balanced = pd.concat([df_minority_undersampled, df_majority_undersampled])

# Shuffle the dataset to mix the classes
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Check new class distribution
print(df_balanced['Label'].value_counts())




Label
Normal    126430
Attack     10139
Name: count, dtype: int64
Label
Attack    100
Normal    100
Name: count, dtype: int64


In [7]:
df=df_balanced
df.drop(columns=['generated', 'startDateTime', 'stopDateTime', 'source', 'destination'], inplace=True)


In [8]:
df

,appName,totalSourceBytes,totalDestinationBytes,totalDestinationPackets,totalSourcePackets,direction,sourceTCPFlagsDescription,destinationTCPFlagsDescription,protocolName,sourcePort,destinationPort,Label
0,Unknown_TCP,64,64,1,1,L2L,S,"R,A",tcp_ip,34431,1971,Attack
1,NetBIOS-IP,64,0,0,1,L2L,S,NaN,tcp_ip,34432,139,Attack
2,Unknown_TCP,64,64,1,1,L2L,S,"R,A",tcp_ip,54806,1010,Attack
3,HTTPImageTransfer,430,850,5,5,L2R,"F,S,P,A","F,S,P,A",tcp_ip,2771,80,Normal
4,DNS,92,439,1,1,L2R,NaN,NaN,udp_ip,31627,53,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...
195,HTTPImageTransfer,1223,31966,25,17,L2R,"F,S,P,A","F,S,P,A",tcp_ip,3765,80,Normal
196,Unknown_TCP,64,64,1,1,L2L,S,"R,A",tcp_ip,54806,1123,Attack
197,Unknown_TCP,192,192,3,3,L2L,S,"R,A",tcp_ip,34431,2001,Attack
198,HTTPWeb,523,875,6,7,L2R,"F,S,R,P,A","F,S,P,A",tcp_ip,4479,80,Normal


In [9]:

from sklearn.preprocessing import LabelEncoder
import pandas as pd

# List of binary categorical columns
binary_cols = ['direction', 'sourceTCPFlagsDescription', 'destinationTCPFlagsDescription', 'Label','appName', 'protocolName']
le = LabelEncoder()

# Apply Label Encoding for binary categorical features
for col in binary_cols:
    df[col] = le.fit_transform(df[col])





In [10]:
df

,appName,totalSourceBytes,totalDestinationBytes,totalDestinationPackets,totalSourcePackets,direction,sourceTCPFlagsDescription,destinationTCPFlagsDescription,protocolName,sourcePort,destinationPort,Label
0,22,64,64,1,1,0,5,5,0,34431,1971,0
1,17,64,0,0,1,0,5,8,0,34432,139,0
2,22,64,64,1,1,0,5,5,0,54806,1010,0
3,9,430,850,5,5,1,3,2,0,2771,80,1
4,4,92,439,1,1,1,9,8,1,31627,53,1
...,...,...,...,...,...,...,...,...,...,...,...,...
195,9,1223,31966,25,17,1,3,2,0,3765,80,1
196,22,64,64,1,1,0,5,5,0,54806,1123,0
197,22,192,192,3,3,0,5,5,0,34431,2001,0
198,10,523,875,6,7,1,4,2,0,4479,80,1


In [11]:
import numpy as np


from sklearn.model_selection import train_test_split
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

y = 2*(y-0.5)

x_tr, x_test, y_tr, y_test = train_test_split(X, y, train_size = 0.7,random_state=42)

from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()
x_tr = scaler.fit_transform(x_tr)

x_test = scaler.transform(x_test)
x_test = np.clip(x_test,0,1)









In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype
---  ------                          --------------  -----
 0   appName                         200 non-null    int64
 1   totalSourceBytes                200 non-null    int64
 2   totalDestinationBytes           200 non-null    int64
 3   totalDestinationPackets         200 non-null    int64
 4   totalSourcePackets              200 non-null    int64
 5   direction                       200 non-null    int64
 6   sourceTCPFlagsDescription       200 non-null    int64
 7   destinationTCPFlagsDescription  200 non-null    int64
 8   protocolName                    200 non-null    int64
 9   sourcePort                      200 non-null    int64
 10  destinationPort                 200 non-null    int64
 11  Label                           200 non-null    int64
dtypes: int64(12)
memory usage: 18.9 KB


In [13]:


import pennylane as qml
import numpy as np

def create_kernel_circuit(nqubits=11):
    dev = qml.device("lightning.qubit", wires=nqubits)
    
    @qml.qnode(dev)
    def kernel_circ(a, b):
        qml.AngleEmbedding(a, wires=range(nqubits))
        qml.adjoint(qml.AngleEmbedding(b, wires=range(nqubits)))
        return qml.probs(wires=range(nqubits))
    
    return kernel_circ



from tqdm import tqdm

from joblib import Parallel, delayed

def qkernel_optimised_2(A, B):
    """
    Compute the Gram matrix using kernel_circ in parallel.
    
    If A and B are square (same number of rows, and identical arrays), then only the
    lower triangle (including diagonal) is computed, and the upper triangle is filled
    by symmetry.
    
    If A and B are not square, the full matrix is computed.
    """
    # Check if both inputs have the same number of rows.
    if A.shape[0] == B.shape[0] and np.array_equal(A, B):
        n = A.shape[0]
        K = np.zeros((n, n))
        
        # Define a function to compute a single kernel value for the pair (i, j)
        def compute_pair(i, j):
            my_kernel = create_kernel_circuit()

            value = my_kernel(A[i], A[j])[0]
            return i, j, value

        # Create list of index pairs for lower triangle (including diagonal)
        indices = [(i, j) for i in range(n) for j in range(i + 1)]
        

        # Create a progress bar for the parallel computation
        results = Parallel(n_jobs=-1)(
            delayed(compute_pair)(i, j) for i, j in tqdm(indices, desc="Computing kernel values")
        )
        
        # Fill in the Gram matrix symmetrically
        for i, j, value in results:
            K[i, j] = value
            if i != j:
                K[j, i] = value
        return K

    else:
        # For non-square inputs or non-identical A and B, compute the full matrix.
        def compute_row(a):
            my_kernel = create_kernel_circuit()
            return [my_kernel(a, b)[0] for b in B]
        
        K = Parallel(n_jobs=-1)(delayed(compute_row)(a) for a in tqdm(A, desc="Computing rows"))
        return np.array(K)


In [14]:
import time
from sklearn.svm import SVC

# Start the timer
start_time = time.time()

# Fit the SVM model
svm = SVC(kernel=qkernel_optimised_2).fit(x_tr, y_tr)

# Calculate and print the fit time
fit_time = time.time() - start_time
print(f"Fit time: {fit_time:.4f} seconds")


/Users/akshaymurthy/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/akshaymurthy/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/akshaymurthy/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/akshaymurthy/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the '

Fit time: 15.9389 seconds


In [15]:
from sklearn.metrics import classification_report,accuracy_score

# Start the timer
start_time = time.time()

# Make predictions
y_pred = svm.predict(x_test)


fit_time = time.time() - start_time
print(f"Predict time: {fit_time:.4f} seconds")

# Generate classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)


Computing rows: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:06<00:00,  8.98it/s]


Predict time: 8.4084 seconds
Classification Report:
              precision    recall  f1-score   support

        -1.0       0.97      1.00      0.98        32
         1.0       1.00      0.96      0.98        28

    accuracy                           0.98        60
   macro avg       0.98      0.98      0.98        60
weighted avg       0.98      0.98      0.98        60



In [ ]:
print("x_tr shape:", x_tr.shape)  # Should be (samples, features)
print("y_tr shape:", y_tr.shape)  # Should be (samples,)

In [ ]:
import pickle
# Save the model to a file
with open('QSVM_ANG_4K.pkl', 'wb') as file:
    pickle.dump(svm, file)